In [1]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [2]:
#spark.sparkContext.stop()

In [3]:
spark = SparkSession \
   .builder \
   .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") \
   .config("spark.sql.catalog.iceberg.catalog-impl", "org.apache.iceberg.gcp.biglake.BigLakeCatalog") \
   .config("spark.sql.catalog.iceberg.gcp_project", "jskuratowicz-477-2023012009191") \
   .config("spark.sql.catalog.iceberg.gcp_location", "us") \
   .config("spark.sql.catalog.iceberg.blms_catalog", "iceberg") \
   .config("spark.sql.catalog.iceberg.warehouse", "gs://bigspark-jskuratowicz-477-2023012009191-yp9htbenkc") \
   .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
   .appName("BigLake Iceberg Example") \
   .enableHiveSupport() \
   .getOrCreate()

23/08/17 13:26:12 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [26]:
#spark.sparkContext.addPyFile("gs://spark-lib/biglake/biglake-catalog-iceberg1.2.0-0.1.0-with-dependencies.jar")

In [27]:
#spark.sql("DROP NAMESPACE IF EXISTS iceberg;")

In [28]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS iceberg;")

DataFrame[]

In [29]:
spark.sql("CREATE DATABASE IF NOT EXISTS jsktest;")

DataFrame[]

In [8]:
spark.sql("SHOW NAMESPACES;").toPandas()

,namespace
0,default
1,iceberg
2,jsktest


In [9]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS iceberg.jsktest;")

DataFrame[]

In [10]:
spark.sql("DESCRIBE NAMESPACE iceberg.jsktest;").toPandas()

,info_name,info_value
0,Namespace Name,jsktest
1,Location,gs://bigspark-jskuratowicz-477-2023012009191-y...
2,Owner,root


In [11]:
spark.sql("SHOW NAMESPACES IN iceberg;").toPandas()

,namespace
0,jsktest


In [48]:
%%bigquery
drop table if exists iceberg_test.trips

Query is running:   0%|          |

""


In [49]:
spark.sql("DROP TABLE IF EXISTS  iceberg.jsktest.trips;")

DataFrame[]

In [50]:
spark.sql("CREATE TABLE IF NOT EXISTS iceberg.jsktest.trips (tripduration bigint, start_station_name string, end_station_name string) \
USING iceberg TBLPROPERTIES(bq_table = 'iceberg_test.trips', bq_connection = 'projects/jskuratowicz-477-2023012009191/locations/us/connections/cloud-iceberg-connect');")

[Thread-5] INFO org.apache.iceberg.gcp.biglake.MigrationManager - Run BigQuery query: "CREATE EXTERNAL TABLE `iceberg_test.trips` WITH CONNECTION `projects/jskuratowicz-477-2023012009191/locations/us/connections/cloud-iceberg-connect` OPTIONS(format='ICEBERG', uris=['blms://projects/jskuratowicz-477-2023012009191/locations/us/catalogs/iceberg/databases/jsktest/tables/trips']);"


DataFrame[]

In [43]:
spark.sql("DESCRIBE TABLE iceberg.jsktest.trips;").toPandas()

,col_name,data_type,comment
0,tripduration,bigint,
1,start_station_name,string,
2,end_station_name,string,
3,,,
4,# Partitioning,,
5,Not partitioned,,


## Lets collect data
Using read API to collect data for our iceberg table

In [31]:
bq_table = spark.read.format('bigquery') \
  .option('table', 'bigquery-public-data.new_york_citibike.citibike_trips') \
  .load()

In [17]:
bq_table.createOrReplaceTempView("bq_temp_table")

In [18]:
spark.sql("select * from bq_temp_table limit 10;")

DataFrame[tripduration: bigint, starttime: string, stoptime: string, start_station_id: bigint, start_station_name: string, start_station_latitude: double, start_station_longitude: double, end_station_id: bigint, end_station_name: string, end_station_latitude: double, end_station_longitude: double, bikeid: bigint, usertype: string, birth_year: bigint, gender: string, customer_plan: string]

In [ ]:
spark.sql("DESCRIBE TABLE bq_temp_table;")

## Write to Iceberg table using Spark
Using metastore definition to append to iceberg table

In [45]:
spark.table("bq_temp_table").select("tripduration","start_station_name","end_station_name").writeTo("iceberg.jsktest.trips").append()

In [46]:
%%bigquery
select count(*) from iceberg_test.trips

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,58937715


In [47]:
spark.sql("select count(*) from bq_temp_table").show()

+--------+
|count(1)|
+--------+
|58937715|
+--------+

